In [2]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

import gym

from env_class import BatteryManagementEnv

plt.style.use("ggplot")

In [3]:
env = BatteryManagementEnv()
env.reset()

array([50.], dtype=float32)

In [4]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space 1
State Space 1


In [5]:
from keras.layers import Dense, Input
from keras.models import Model



# Assuming the observation space includes the last 24 actions
input_shape = 1  # Current charge level
model_input = Input(shape=(input_shape))
#x = Flatten()(model_input)
hidden_layer = Dense(24, activation='relu')(model_input)
output = Dense(env.nA, activation='relu')(hidden_layer)  # Each action has 3 possible values

model = Model(inputs=model_input, outputs=output)

In [6]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()
from tensorflow.keras.optimizers.legacy import Adam

# Then, define DQN agent in Keras-RL
memory = SequentialMemory(limit=2000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy, 
               nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [7]:
dqn.fit(env, nb_steps=1000, visualize=False, verbose=1, nb_max_episode_steps=200, log_interval=10000)

Training for 1000 steps ...
Interval 1 (0 steps performed)




ValueError: Error when checking input: expected input_1 to have 2 dimensions, but got array with shape (1, 1, 1)